In [1]:
import torch
from torchvision import models
import torch.nn as nn
import copy

In [2]:
dropout_rate = 0.4

In [3]:
def set_parameter_requires_grad(model, feature_extracting, num_layers_to_train):
    ## Versione per introdurre Dropout2d intermedi frai vari Conv2d
    if feature_extracting:
        t = 0
        for child in model.modules():
            if isinstance(child,nn.Conv2d):
                t+=1
                
        print(f'Conv2d layers re-trained in the model: {num_layers_to_train}/{t}')
        
        c = 0
        c_dropout = 0
        is_first_time = True
        
        #
        for name,child in model.named_modules():
            splits = name.split('.')
            
            if is_first_time:
                is_first_time = False
                ## TODO: Versione tutta congelata e alcuni decongelati
                for param in child.parameters(): #first, freeze all the layers from the top
                   
                    param.requires_grad = False

            if isinstance(child,nn.Conv2d):
                c+=1
            
            assert(t>=num_layers_to_train)
            
            if c > t - num_layers_to_train: #un-freeze all the following layers (conv2d & bn)
                for param in child.parameters():
                    param.requires_grad = True 
       
        model_copy = copy.deepcopy(model)
        
        for name,child in model_copy.named_modules():
            splits = name.split('.')

                
            if isinstance(child,nn.Conv2d):
                c+=1
            
            assert(t>=num_layers_to_train)
            
            if c > t - num_layers_to_train: #un-freeze all the following layers (conv2d & bn)
                    
                if isinstance(child,nn.Conv2d) and splits[-1]=='conv1': #TODO conv1
                   
                    c_dropout += 1
                    
                    # if c_dropout <= 3: #TODO i primi 3
                    #     new_module = nn.Sequential(
                    #             child,
                    #             nn.Dropout2d(p=dropout_rate))
    
                    #     if len(splits)==1:
                    #         setattr(model, name, new_module)
                    #     elif len(splits)==3:
                    #         setattr(getattr(model,splits[0])[int(splits[1])], splits[2], new_module)
                    #     # #
                    #     # elif len(splits)==4:
                    #     #     setattr(getattr(getattr(model,splits[0])[int(splits[1])],splits[2]), splits[3], new_module)
                    
                    if c_dropout > 9 - 3: # hardcode 9 e 3; TODO gli ultimi 3 (9 è il numero totale di conv1)
                        new_module = nn.Sequential(
                                child,
                                nn.Dropout2d(p=dropout_rate))
    
                        if len(splits)==1:
                            setattr(model, name, new_module)
                        elif len(splits)==3:
                            setattr(getattr(model,splits[0])[int(splits[1])], splits[2], new_module)      
      

In [90]:
def set_parameter_requires_grad_alexnet(model, feature_extracting, num_layers_to_train):
    ## Versione per introdurre Dropout2d intermedi frai vari Conv2d
    if feature_extracting:
        t = 0
        for child in model.modules():
            if isinstance(child,nn.Conv2d):
                t+=1
                
        print(f'Conv2d layers re-trained in the model: {num_layers_to_train}/{t}')
        
        c = 0
        c_dropout = 0
        is_first_time = True
        
        #
        for name,child in model.named_modules():
            splits = name.split('.')
            
            if is_first_time:
                is_first_time = False
                ## TODO: Versione tutta congelata e alcuni decongelati
                for param in child.parameters(): #first, freeze all the layers from the top
                   
                    param.requires_grad = False
                
            if isinstance(child,nn.Conv2d):
                c+=1
            
            assert(t>=num_layers_to_train)
            
            if c > t - num_layers_to_train: #un-freeze all the following layers (conv2d & bn)
                for param in child.parameters():
                    param.requires_grad = True 

        model_copy = copy.deepcopy(model)
        
        for name,child in model_copy.named_modules():
            splits = name.split('.')

                
            if isinstance(child,nn.Conv2d):
                c+=1
            
            assert(t>=num_layers_to_train)
            
            if c > t - num_layers_to_train: #un-freeze all the following layers (conv2d & bn)
                    
                if isinstance(child,nn.Conv2d): #TODO conv1
                   
                    c_dropout += 1
                    
                    # if c_dropout <= 3: #TODO i primi 3
                    #     new_module = nn.Sequential(
                    #             child,
                    #             nn.Dropout2d(p=dropout_rate))
    
                    #     if len(splits)==1:
                    #         setattr(model, name, new_module)
                    #     elif len(splits)==3:
                    #         setattr(getattr(model,splits[0])[int(splits[1])], splits[2], new_module)
                    #     # #
                    #     # elif len(splits)==4:
                    #     #     setattr(getattr(getattr(model,splits[0])[int(splits[1])],splits[2]), splits[3], new_module)
                    
                    if c_dropout > 5 - 3: # hardcode 9 e 3; TODO gli ultimi 3 (9 è il numero totale di conv1)
                        new_module = nn.Sequential(
                                child,
                                nn.Dropout2d(p=dropout_rate))
    
                        # if len(splits)==1:
                        #     setattr(model, name, new_module)
                        if len(splits)==2:
                            # print(f'name={name} --- \n\tgetattr={getattr(model,splits[0])} --- \n\t\tgetattr[0]={getattr(model,splits[0])[int(splits[1])]}')
                            
                            # setattr(model, splits[1], new_module)
                            # setattr(model, splits[0], new_module)
                            # setattr(model, name, new_module)
                            setattr(model.features, splits[1], new_module)

                            # setattr(getattr(model,splits[0])[int(splits[0])], splits[1], new_module)
                        # elif len(splits)==3:
                        #     setattr(getattr(model,splits[0])[int(splits[1])], splits[2], new_module)      
      

In [64]:
use_pretrained = True

In [87]:
model_alexnet = models.alexnet(pretrained=use_pretrained)

/Users/andreaberti/miniconda3/envs/torchgpu/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/andreaberti/miniconda3/envs/torchgpu/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [88]:
model_alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [93]:
model_alexnet._get_name()

'AlexNet'

In [94]:
model_resnet18 = models.resnet18(pretrained=use_pretrained)

/Users/andreaberti/miniconda3/envs/torchgpu/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/andreaberti/miniconda3/envs/torchgpu/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [95]:
model_resnet18._get_name()

'ResNet'

In [11]:
model_resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
for name, child in model_alexnet.named_modules():
    print(f'name={name} --- child={child}')

name= --- child=AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=40

In [13]:
for name, child in model_resnet18.named_modules():
    print(f'name={name} --- child={child}')

name= --- child=ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(

In [91]:
set_parameter_requires_grad_alexnet(model_alexnet, True, num_layers_to_train=5)

Conv2d layers re-trained in the model: 5/5


In [92]:
model_alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Sequential(
      (0): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Dropout2d(p=0.4, inplace=False)
    )
    (7): ReLU(inplace=True)
    (8): Sequential(
      (0): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Dropout2d(p=0.4, inplace=False)
    )
    (9): ReLU(inplace=True)
    (10): Sequential(
      (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Dropout2d(p=0.4, inplace=False)
    )
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, 

In [85]:
for name, child in model_alexnet.named_modules():
    print(f'name={name} --- child={child}')

name= --- child=AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=40